1. DataLoader를 이용해 MNIST 데이터셋 로드

In [9]:
import torch
import torch.nn as nn
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

training_epochs = 15  # training 반복 횟수
batch_size = 100

root = './data'
mnist_train = dset.MNIST(root=root, train=True, transform=transforms.ToTensor(), download=True)
mnist_test = dset.MNIST(root=root, train=False, transform=transforms.ToTensor(), download=True)

# data loader를 구현
train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=False)

2. 데이터가 준비되었다면, 이젠 그 데이터를 학습할 모델을 구현할 차례입니다. 그 후 모델 안의 가중치를 초기화시켜보세요. 입력 데이터 형태에 맞도록 linear한 모델을 구성해보세요.

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
linear = torch.nn.Linear(784, 10, bias=True).to(device)
torch.nn.init.normal_(linear.weight)  # weight 초기화

Parameter containing:
tensor([[-2.5395,  1.1449, -0.7209,  ..., -0.8589, -2.8663, -1.2511],
        [ 0.9226,  0.6787,  1.3152,  ..., -0.2271,  1.0642, -0.0929],
        [ 1.1027, -0.1464, -2.0619,  ...,  0.2932, -0.2545,  1.6537],
        ...,
        [-0.4417, -0.1979,  0.4105,  ...,  0.1739, -0.8267,  0.5866],
        [-1.7455, -0.1817, -0.7749,  ...,  0.2513,  0.4484, -0.7206],
        [ 0.0127,  0.8570,  0.3458,  ..., -1.8873, -3.0059, -0.9351]],
       requires_grad=True)

3. 아래 제시된 loss 함수와 optimizer를 구현해보세요.

In [5]:
# Loss fn - Cross Entropy Loss
criterion = torch.nn.CrossEntropyLoss().to(device)

# optimizer - SGD
optimizer = torch.optim.SGD(linear.parameters(), lr=0.1)

4. 모델, optimizer, loss fn 등을 이용해 학습 구현하기

In [10]:
for epoch in range(training_epochs):
  for i, (imgs, labels) in enumerate(train_loader): # 리스트 순서, value 전달
    imgs, labels = imgs.to(device), labels.to(device)
    imgs = imgs.view(-1, 28*28)

    outputs = linear(imgs)
    loss = criterion(outputs, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    _, argmax = torch.max(outputs, 1)
    accuracy = (labels == argmax).float().mean()

    if (i+1) % 100 == 0:
      print('Epoch [{}/{}], Steop [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'.format(
          epoch+1, training_epochs, i+1, len(train_loader), loss.item(), accuracy.item()*100))

Epoch [1/15], Steop [100/600], Loss: 5.1447, Accuracy: 33.00%
Epoch [1/15], Steop [200/600], Loss: 2.3513, Accuracy: 51.00%
Epoch [1/15], Steop [300/600], Loss: 2.3606, Accuracy: 63.00%
Epoch [1/15], Steop [400/600], Loss: 1.4759, Accuracy: 69.00%
Epoch [1/15], Steop [500/600], Loss: 1.0778, Accuracy: 77.00%
Epoch [1/15], Steop [600/600], Loss: 0.8432, Accuracy: 77.00%
Epoch [2/15], Steop [100/600], Loss: 0.6664, Accuracy: 83.00%
Epoch [2/15], Steop [200/600], Loss: 1.1041, Accuracy: 74.00%
Epoch [2/15], Steop [300/600], Loss: 0.8450, Accuracy: 84.00%
Epoch [2/15], Steop [400/600], Loss: 1.5385, Accuracy: 76.00%
Epoch [2/15], Steop [500/600], Loss: 1.4800, Accuracy: 73.00%
Epoch [2/15], Steop [600/600], Loss: 0.9173, Accuracy: 78.00%
Epoch [3/15], Steop [100/600], Loss: 1.0738, Accuracy: 81.00%
Epoch [3/15], Steop [200/600], Loss: 1.1980, Accuracy: 77.00%
Epoch [3/15], Steop [300/600], Loss: 0.9031, Accuracy: 83.00%
Epoch [3/15], Steop [400/600], Loss: 0.5138, Accuracy: 84.00%
Epoch [3

5. 테스트 데이터로 테스트 진행

In [11]:
linear.eval()
with torch.no_grad():
  correct=0
  total=0
  for i, (imgs, labels) in enumerate(test_loader):
    imgs, labels = imgs.to(device), labels.to(device)
    imgs = imgs.view(-1, 28*28)
    outputs = linear(imgs)
    _, argmax = torch.max(outputs, 1)  # amx함수 통해 최종 출력이 가장 높은 class 선택
    total += imgs.size(0)
    correct += (labels == argmax).sum().item()

  print('Test accuracy for {} images: {:.2f}%'.format(total, correct / total * 100))

Test accuracy for 10000 images: 89.18%
